# TT data analytics by RV ver1.0

In [63]:
#from IPython.display import Javascript
#Javascript("Jupyter.notebook.execute_cells([2,3])")
from IPython.display import IFrame

IFrame(src='https://rikvalentini.wixsite.com/naturetalkers', width=1000, height=1000)

# INPUT YOUR TT CLOUD NUMBER AND OPERATION MODE
> loop ON mode if you want to automatic updates every hour your data files, otherwise for single run do not check the box

In [61]:

import ipywidgets as widgets
from ipywidgets import FileUpload
from IPython.display import Javascript

TT_cloud=widgets.Text('Input TTCLoud serial number')
display (TT_cloud)

loop_on=widgets.Checkbox(
    value=False,
    description='Loop ON',
    disabled=False,
    indent=False
)

display (loop_on)
time_cycle=widgets.Text('time cycle in seconds')
display (time_cycle)



Text(value='Input TTCLoud serial number')

Checkbox(value=False, description='Loop ON', indent=False)

Text(value='time cycle in seconds')

# INPUT YOUR TTs SERIAL NUMBERS
>Prepare an excel file with one column containing the TTs serial numbers 
>with the column name 'ID' and save as .csv file (i.e C0200114.csv)- The directory of this file will be the repositiry of all the elaborated data for this Cloud



|  ID  |
|-|
|   52060652  |
|   52060669  |
|   52060671  |
|   52060656  |
|   52060668  |
|   52060664  |
|   52060667  |

In [5]:
from ipyfilechooser import FileChooser
fc = FileChooser('/Users/')
display(fc)
# fc.selected  is file path

FileChooser(path='/Users', filename='', title='HTML(value='', layout=Layout(display='none'))', show_hidden='Fa…

# Input the Email addresses to send TTs battery report 
###### insert email addresses separated by commas (i.e. rik@unitus.it, rikvalentini@gmail.com,....)
> (skip it if you do not need this function)

In [42]:
mail_addr_from=widgets.Text(description='FROM',value='your mail')
display (mail_addr_from)
mail_addr_to=widgets.Text(description='TO',value='i.e. rik@unitus.it,...')
display (mail_addr_to)
# mail_list = mail_addr.value.split(',') # this create a list of mail addresses

mail_smtp=widgets.Text(description='SMTP server',value='i.e. smtp.gmail.com')
display (mail_smtp)
mail_port=widgets.Text(description='SMTP port',value='i.e. 465')
display (mail_port)
mail_pw=widgets.Text(description='Password',value='')
display (mail_pw)

mail_on=widgets.Checkbox(
    value=False,
    description='Send mail',
    disabled=False,
    indent=False
)

mail_store=widgets.Checkbox(
    value=False,
    description='Store mail addr',
    disabled=False,
    indent=False
)


display (mail_on)
display (mail_store)

Text(value='your mail', description='FROM')

Text(value='i.e. rik@unitus.it,...', description='TO')

Text(value='i.e. smtp.gmail.com', description='SMTP server')

Text(value='i.e. 465', description='SMTP port')

Text(value='', description='Password')

Checkbox(value=False, description='Send mail', indent=False)

Checkbox(value=False, description='Store mail addr', indent=False)

In [71]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import time
import math
from ftplib import FTP
import csv
import dask.dataframe as dd
%matplotlib inline
import matplotlib.pyplot as plt
import smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders  
# Set it to None to display all columns in the dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5000)
pd.options.mode.chained_assignment = None 



#Data transformation in physical units
def transf():
    dfn['Tref_0c']=127.6-0.006045*dfn['Tref_0'] + 1.26E-07*dfn['Tref_0']**2 -1.15E-12*dfn['Tref_0']**3
    dfn['Tref_1c']=127.6-0.006045*dfn['Tref_1'] + 1.26E-07*dfn['Tref_1']**2 -1.15E-12*dfn['Tref_1']**3
    dfn['Theat_0c']=127.6-0.006045*dfn['Theat_0'] + 1.26E-07*dfn['Theat_0']**2 -1.15E-12*dfn['Theat_0']**3
    dfn['Theat_1c']=127.6-0.006045*dfn['Theat_1'] + 1.26E-07*dfn['Theat_1']**2 -1.15E-12*dfn['Theat_1']**3
    #dfn['growth_sensor_c']=194.856731-0.008274771*dfn.growth_sensor+1.63685E-07*dfn.growth_sensor**2-1.58251E-12*dfn.growth_sensor**3+ 5.96455E-18*dfn.growth_sensor**4
    dfn['growth_sensor_c']=-0.0006*dfn['growth_sensor']+61.079
    dfn['angle_mean_c']=np.arctan((dfn['g_xmean']**2+(dfn['g_ymean']**2)/dfn['g_zmean'])**0.5)*90/(np.pi/2)
    dfn['trunk_axis_movement']=dfn['g_xstd']+dfn['g_ystd']+dfn['g_zstd']
    dfn['Vbat_c']= round(2*1100*dfn['adc_Vbat']/dfn['adc_bandgap']/1000,2)
    dfn['date'] = pd.to_datetime(dfn['timestamp'],unit='s')
    #dfn['date'].dt.tz_localize('utc').dt.tz_convert('Europe/Rome') in case of usimng EU time
    dfn['date'] = dfn['date'].dt.strftime('%d/%m/%Y/%H:%M:%S')
    dfn['air_temperature'] = dfn['air_temperature']/10

def transf_ff():
    dfn1['air_temperature_ff'] = dfn1['air_temperature_ff']/10
    dfn1['gas_co2']=dfn1['gas_co2']*1.25-500. # see MHZ19B datasheet 0-2000 ppm --> 400 - 2000 mVolts
    dfn1['gas_o3']=dfn1['gas_o3']*-0.076+390 # see mq131 (low concebration manual) 10 - 200 ppb  ---> 5000 - 2500 mVolts
    dfn1['gas_o3']=dfn1['gas_o3']*48/24.45 # conversion in ug m-3
    #dfn1['leaf_temperature']=dfn1['leaf_temperature']/210 -273.15 # conversione 210 counts/K
    dfn1['leaf_temperature']=25+(dfn1['leaf_temperature']-64500)/210  # conversione 210 counts/K con offset di 64500@25°C see excelitas data sheet TPS 1S 1051
    dfn1['air_temperature_IR']=25+ (dfn1['air_temperature_IR']-8200)/90  # conversione 90 counts/K con offset di 8200@25°C
    dfn1['date'] = pd.to_datetime(dfn1['timestamp'],unit='s')
    #dfn['date'].dt.tz_localize('utc').dt.tz_convert('Europe/Rome') in case of usimng EU time
    
    

#Showing batteries voltage
def Vbat_show():
    print (names_tt[i],dfn.loc[len(dfn)-1,'Vbat_c'],dfn.loc[len(dfn)-1,'date'])
    #dt.datetime.fromtimestamp(1596725594).strftime('%Y-%m-%d %H:%M:%S %j' )

#Sending report check
def Vbat_mail():
    
    
    
    
    toaddr = data_mail[1].split(',') # this create a list of mail addresses
    fromaddr = data_mail[0]
    smtp=data_mail[2]
    port=data_mail[3]
    pw=data_mail[4]
    
    
     # instance of MIMEMultipart 
    msg = MIMEMultipart() 
      
    # storing the senders email address   
    msg['From'] = fromaddr 
      
    # storing the receivers email address  
  
    msg['To'] = ', '.join(toaddr)
    # storing the subject  
    msg['Subject'] = cloudID_tt
      
    # string to store the body of the mail 
    body = "Battery test"
      
    # attach the body with the msg instance 
    msg.attach(MIMEText(body, 'plain')) 
      
    # open the file to be sent  
    filename = url_proj+'/'+cloudID_tt+'_BAT_'+day+'.csv'
    attachment = open(filename, "rb") 
      
    # instance of MIMEBase and named as p 
    p = MIMEBase('application', 'octet-stream') 
      
    # To change the payload into encoded form 
    p.set_payload((attachment).read()) 
      
    # encode into base64 
    encoders.encode_base64(p) 
       
    p.add_header('Content-Disposition', "attachment; filename= %s" % cloudID_tt+'_BAT_'+day+'.csv') 
      
    # attach the instance 'p' to instance 'msg' 
    msg.attach(p) 
      
    # creates SMTP session 
    s = smtplib.SMTP_SSL(smtp, port) 
      
    # start TLS for security 
    #s.starttls() 
      
    # Authentication 
    s.login(fromaddr, pw) 
      
    # Converts the Multipart msg into a string 
    text = msg.as_string() 
      
    # sending the mail 
    s.sendmail(fromaddr, toaddr, text) 
      
    # terminating the session 
    s.quit() 

def elab_sapflow():
    
    #do equation
    df_night=dfn[dfn.a_650<=10]
    df_night['date_night']=pd.to_datetime(df_night['timestamp'],unit='s')
    df_night['date_night'] = df_night['date_night'].dt.strftime('%d%m%Y')
    df_night['dTon']= df_night['Theat_1c']-df_night['Tref_1c']
    df_night ['dToff']=df_night['Theat_0c']-df_night['Tref_0c']
    df_night['dTindex']=df_night['dTon']-df_night['dToff']
    
    
    dfn['dTon']= dfn['Theat_1c']-dfn['Tref_1c']
    dfn['dToff']=dfn['Theat_0c']-dfn['Tref_0c']
    dfn['dTindex']=dfn['dTon']-dfn['dToff']
    
    for j in range (0,len(dfn)):
            
            day_n=pd.to_datetime(dfn.loc[j,'timestamp'],unit='s').strftime('%d%m%Y')
            #.strftime('%d%m%Y')
            df_day=df_night[df_night['date_night']==day_n]
            dTmax=df_day['dTindex'].max()
            dfn.loc[j,'sap_flow_density_raw']=12.95*((dTmax/(dfn.loc[j,'dTindex'])-1))*10 #transf from l dm-2 h-1 to g cm-2 h-1
            if dfn.loc[j,'sap_flow_density_raw'] <0:
                dfn.loc[j,'sap_flow_density_raw']=0
    
    dfn['sap_flow_density']=dfn.loc[0:len(dfn),'sap_flow_density_raw'].rolling(window=4).mean() # rolling mean of interpolated data
    
    # granier equation
    #dfn['dTindex']=dfn['Theat_1c']-dfn['Tref_1c']
    #for j in range (25,len(dfn)):
        #dTmax=dfn.loc[j-25:j,'dTindex'].max()
        #dfn.loc[j,'sap_flow_density']=118.99*10000*((dTmax/(dfn.loc[j,'dTindex'])-1))/3600 #transf from g m-2 s-1 to g cm-2 h-1
        
    dfn['stem_humidity']=dfn.freq # to be upgraded by shahla
    
    

def elab_growth():
    
    dfn_night=dfn[dfn['a_650']==0]# select only night values
    dfn['growth_sensor_night']=dfn_night['growth_sensor_c']
    avg_g=dfn['growth_sensor_night'].median()# calculate median
    std_g=dfn['growth_sensor_night'].std()#calculate std
    
    for j in range (0,len(dfn)):
            if dfn.loc[j,'growth_sensor_night']>avg_g+2*std_g or dfn.loc[j,'growth_sensor_night']<avg_g-2*std_g: # correct for numbers outside +- 2*std
                dfn.loc[j,'growth_sensor_night']=avg_g #replace outliers with median 
    dfn.growth_sensor_night.interpolate(method='pad',axis=0, inplace=True) #interpolation of missing data NaN
    dfn['growth_roll']=dfn.loc[0:len(dfn),'growth_sensor_night'].rolling(window=200).mean() # rolling mean of interpolated data
    dfn['growth_roll_diff']=dfn['growth_roll'].diff() # derivative (growth) of the distance
    dfn.loc[dfn.growth_roll_diff>0,'growth_roll_diff']=0  #to be applied in case we want non-negative growth
    
    dfn['radial_growth']=-dfn['growth_roll_diff'].cumsum() # cumulative growth (negative since growth equal less distance from trunk)
    
    
    

def elab_stem_humidity():
    #dfn['stem_humidity']=((dfn.freq-(-74.149*dfn['Tref_0c']) + 33165)*-0.0001+1.9581)*100 # according to shahla
    a=-72.768*dfn['Tref_0c'] + 27392 # probe in water sensitivity
    dfn['stem_humidity']=((1-(dfn.freq-a)/a))*100 # this is as reference to water 100%
      

def elab_light ():
   

    dfn.loc[dfn.a_610 >65532,'a_610']=np.nan
    dfn.loc[dfn.a_680 >65532,'a_680']=np.nan
    dfn.loc[dfn.a_730 >65532,'a_730']=np.nan
    dfn.loc[dfn.a_760 >65532,'a_760']=np.nan
    dfn.loc[dfn.a_810 >65532,'a_810']=np.nan
    dfn.loc[dfn.a_860 >65532,'a_860']=np.nan
    dfn.loc[dfn.a_450 >65532,'a_450']=np.nan
    dfn.loc[dfn.a_500 >65532,'a_500']=np.nan
    dfn.loc[dfn.a_550 >65532,'a_550']=np.nan
    dfn.loc[dfn.a_570 >65532,'a_570']=np.nan
    dfn.loc[dfn.a_600 >65532,'a_600']=np.nan
    dfn.loc[dfn.a_650 >65532,'a_650']=np.nan
    
    dfn['ndvi_raw']=(dfn.a_760-dfn.a_680)/(dfn.a_760+dfn.a_680)
    dfn.ndvi_raw.interpolate(method='pad',axis=0, inplace=True) #interpolation of missing data NaN
    dfn.loc[dfn.ndvi_raw <0,'ndvi_raw']=0
    dfn['ndvi']=dfn.loc[0:len(dfn),'ndvi_raw'].rolling(window=4).mean() # rolling mean of interpolated data
    
    dfn['green_red_raw']=(dfn.a_570-dfn.a_680)/(dfn.a_570+dfn.a_680)
    dfn.green_red_raw.interpolate(method='pad',axis=0, inplace=True) #interpolation of missing data NaN
    dfn.loc[dfn.green_red_raw <0,'green_red_raw']=0
    dfn['green_red']=dfn.loc[0:len(dfn),'green_red_raw'].rolling(window=4).mean() # rolling mean of interpolated data
    


url_proj= fc.selected_path

    # Store mail parameters
mail_list=mail_addr_to.value.split(',')
data_mail=[mail_addr_from.value,mail_addr_to.value,mail_smtp.value,mail_port.value,mail_pw.value]

if mail_store.value:
    
    with open (url_proj+'/'+'mail_params.csv','w') as f:
               
                data_writer = csv.writer(f, delimiter=',')
                data_writer.writerow(data_mail)

if mail_on.value and mail_pw.value =='':
    
    
    with open (url_proj+'/'+'mail_params.csv','r') as f:
        reader = csv.reader(f, delimiter=',')
        data_mail = next(reader) 


    #READ THE CLOUD    

flag_bat=1
cloudID_tt=TT_cloud.value


#TTcloud_tt=url_site+cloudID_tt

cols=['timestamp','tree_talker', 'sap_flow_density','stem_humidity','radial_growth','ndvi','green_red','air_temperature','rel_air_humidity','trunk_axis_movement']

#dfn_cum=pd.DataFrame(columns=cols) 
dfn_day=pd.DataFrame(columns=cols) 

d=dt.datetime.now()
old_day=d.strftime('%d.%m.%Y')
        
        
while True:
  
        
        t0=time.time()
        ts = str(int(time.time()))
       
       
        
        
        while True :
            try:
                    print ("job start at ... ",dt.datetime.now())
                    url_tt = 'http://naturetalkers.altervista.org/'+cloudID_tt+'/ttcloud.txt'
                    df_tt=pd.read_csv(url_tt, sep=',',engine='python',header=None)
                    
            except:
                    print ("altervista server failure .....")
                    time.sleep(6)
                    pass
            else:
                    break



        
       
        
        
        dtt=pd.read_csv(fc.selected, sep=',',index_col=False, engine='python',header=0)
        dtt.ID=dtt.ID.astype(str)
      
        
        print ('Data retrieval time seconds .....',time.time()-t0)
        t1=time.time()
               
        #CREATE DATAFRAME FOR TT AND TT FIRE
        
        a=[]
        col0=df_tt.iloc[:,0].to_list()
        col1=df_tt.iloc[:,1].to_list() 
        for i in range (len(col1)):
                a.append(col1[i].split(';'))
        dfraw=pd.DataFrame(a)
        df_tt=dfraw[480*31:len(dfraw)] # we take data after 240 hours of stabilizatio (10 days)
        df_tt.reset_index(drop=True,inplace=True)
        
               
               
        
        #TREE TALKER SECTION
        
        #CLEANING DATAFRAME GENERATING COMBINED STRING 49 AND 4D FOR ID of treetalkers
        
      
        names_tt=dtt.ID
        
        
        #dfn_cum.to_csv (url_names+'_'+day+'.csv',index=False)
        df_Vbat=pd.DataFrame(columns=['ID','date','Vbat'])


        for i in range (0,len(dtt)):
                dfn_day=pd.DataFrame(columns=cols)
                dfx=df_tt[df_tt.iloc[:,0]==names_tt[i]]
                if len(dfx)==0:
                    print (names_tt[i], 'missing')
                    row_bat=[names_tt[i],old_day,'missing']                 
                    df_Vbat.loc[len(df_Vbat)]=row_bat
                    continue
            
                dfn1=dfx[dfx.iloc[:,2]=='4D']
                col_delete=[1,2,21,22,23,24,25,26,27,28,29] 
                dfn1.drop(col_delete,axis=1,inplace =True)
                col1=['tree_talker','timestamp','Tref_0','Theat_0','growth_sensor',
                'adc_bandgap','bits','rel_air_humidity','air_temperature','g_zmean','g_zstd','g_ymean','g_ystd','g_xmean','g_xstd','Tref_1','Theat_1','freq','adc_Vbat']
                dfn1.columns=col1
                dfn1.drop_duplicates(subset=None, inplace=True)
                dfn1.reset_index(drop=True, inplace=True)
                
                dfn2=dfx[dfx.iloc[:,2]=='49']
                col_delete=[1,2,18,19,20,21,22,23,24,25,26,27,28,29] 
                dfn2.drop(col_delete,axis=1,inplace =True)
                col2=['tree_talker','timestamp','a_610','a_680','a_730','a_760','a_810','a_860','a_450','a_500','a_550','a_570','a_600','a_650','integration_time','gain']
                dfn2.columns=col2
                dfn2.drop_duplicates(subset=None, inplace=True)
                dfn2.reset_index(drop=True, inplace=True)
                
                dfn=pd.merge(dfn1,dfn2,on=['tree_talker','timestamp'])
                dfn.drop_duplicates(subset=None, inplace=True)
                dfn.reset_index(drop=True, inplace=True)
                dfn.timestamp=dfn.timestamp.astype(int)
                dfn.timestamp=round(dfn.timestamp/10)*10
                for j in dfn.columns[2:32]:
                    dfn[j]=dfn[j].astype(float)
               
                dfn['timestamp']=dfn['timestamp'].astype(int)
                dfn.to_csv(url_proj+'/'+names_tt[i]+'_raw.csv',index=False)
                transf()
                dfn.to_csv(url_proj+'/'+names_tt[i]+'_transf.csv',index=False)
                
               
                Vbat_show()
                elab_sapflow()
                elab_growth()
                elab_stem_humidity()
                elab_light()
                
                row_bat=[dfn.loc[len(dfn)-1,'tree_talker'],dfn.loc[len(dfn)-1,'date'],dfn.loc[len(dfn)-1,'Vbat_c']]          
                df_Vbat.loc[len(df_Vbat)]=row_bat
               
                dfn.to_csv(url_proj+'/'+names_tt[i]+'_elab.csv',index=False)
                
                dfn_c=dfn[['tree_talker', 'timestamp','date','sap_flow_density','stem_humidity','radial_growth','ndvi','green_red','air_temperature','rel_air_humidity','trunk_axis_movement']]
                dfn_c.reset_index(drop=True, inplace=True)
                
            
                d=dt.datetime.now()
                day=d.strftime('%d.%m.%Y')
                last_date = pd.to_datetime(dfn_c.loc[len(dfn_c)-1,'timestamp'],unit='s')
    
                last_date=last_date.strftime('%d/%m/%Y/%H:%M:%S')
                #dfn_c['date']=last_date
                #dfn_cum.loc[len(dfn_cum)]=dfn_tot.loc[len(dfn_tot)-1]
                
               
                dfn_day.loc[len(dfn_day)]=dfn_c.loc[len(dfn_c)-1]
                
                file_day =url_proj+'/'+cloudID_tt+'_'+day
                if os.path.exists(file_day+'.csv'):
                    bol=False
                else:
                    bol=True
                with open(file_day+'.csv','a+') as f:
                           dfn_day.to_csv(f,index=False,header=bol)
                
              
                
        
        
                
      
        df_Vbat.to_csv(url_proj+'/'+cloudID_tt+'_BAT_'+day+'.csv',index=False)
        
        if (day != old_day or flag_bat==1) and mail_on.value:# flag 1 only when you start script for the first time
            Vbat_mail()
            old_day=day
            flag_bat=0
       
        
     
        print ('TT total elaboration time seconds .....',time.time()-t0)
        
        
        
       
        if loop_on.value:
            time.sleep(time_cycle.value)
        else:
            print ('job ended')
            break
            
        print('Waiting next job .......')

job start at ...  2020-09-21 17:24:03.375240
Data retrieval time seconds ..... 6.800644874572754
52060628 3.72 21/09/2020/12:00:00
52060582 3.87 21/09/2020/13:00:00
52060611 3.9 21/09/2020/15:00:00
52060614 missing
52060591 3.81 21/09/2020/14:00:00
52060619 3.78 21/09/2020/06:00:00
52060622 3.83 21/09/2020/13:00:00
52060641 3.95 21/09/2020/15:00:00
52060594 3.76 21/09/2020/15:00:00
52060595 3.74 21/09/2020/12:00:00
52060584 3.21 17/09/2020/15:00:00
52060596 3.87 21/09/2020/15:00:00
52060607 3.88 21/09/2020/14:00:00
52060593 3.98 21/09/2020/14:00:00
52060597 3.76 21/09/2020/14:00:00
52060600 3.86 21/09/2020/14:00:00
52060616 4.15 06/09/2020/12:00:00
52060569 4.06 15/09/2020/00:00:00
52060589 3.32 28/08/2020/10:00:00
52060637 4.01 09/09/2020/06:00:00
52060604 3.84 21/09/2020/13:00:00
52060577 3.79 21/09/2020/13:00:00
52060571 3.76 21/09/2020/10:00:00
52060573 3.73 21/09/2020/10:00:00
52060648 3.96 16/09/2020/23:00:00
52060620 3.84 20/09/2020/00:00:00
52060575 4.02 13/09/2020/19:00:00
520

In [1]:
pip freeze > requirements.txt